In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

In [ ]:
data = np.loadtxt("E:/nmtuan97/1_scripts/ml_learning/data_test.csv", delimiter=',')
X = data[:,:1]
y = data[:,1:2]

# noise = np.random.uniform(-25, 25, len(y)).reshape(len(y),1)
X = MinMaxScaler().fit_transform(X)
# y = y + noise

plt.plot(X, y, 'o', label='data looks like')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.33)
kernel = 1 * RBF(length_scale=280, length_scale_bounds=(1e-2, 1e2))
gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
gaussian_process.fit(X_train, y_train)
gaussian_process.kernel_

In [ ]:
r2_ = 0.99
r2_train_ = 0.0
r2_test_ = 0.0
max_ = 0
while r2_test_ < float(r2_):
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.33)
    
    kernel = 1 * RBF(length_scale=280, length_scale_bounds=(1e-2, 1e2))
    gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
    gaussian_process.fit(X_train, y_train)
    
    y_train_predicted_svr_, a = gaussian_process.predict(X_train, return_std=True)
    y_test_predicted_svr_, a = gaussian_process.predict(X_test, return_std=True)
    
    rmse_train_ = np.sqrt(mean_squared_error(y_train, y_train_predicted_svr_))
    r2_train_ = r2_score(y_train, y_train_predicted_svr_)
    mae_train_ = mean_absolute_error(y_train, y_train_predicted_svr_)
    # Evaluating test dataset
    rmse_test_ = np.sqrt(mean_squared_error(y_test, y_test_predicted_svr_))
    r2_test_ = r2_score(y_test, y_test_predicted_svr_)
    mae_test_ = mean_absolute_error(y_test, y_test_predicted_svr_)
    
    if r2_test_ > max_:
        max_ = r2_test_
        print("R2 score of test is {}".format(max_))
        
print("-------------------------------------------")
print("RMSE of training is {}".format(rmse_train_))
print("R2 score of training  is {}".format(r2_train_))
print("MAE of training is {}".format(mae_train_))
print("RMSE of test is {}".format(rmse_test_))
print("R2 score of test is {}".format(max_))
print("MAE of test is {}".format(mae_test_))

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.33)

noise = np.random.uniform(-100, 100, len(y_test)).reshape(len(y_test),1)

X_test = np.sort(X_test, axis=0)

y_test = np.sort(y_test, axis=0) + noise

mean_prediction, std_prediction = gaussian_process.predict(X_test, return_std=True)

# score_ = gaussian_process.score(X,y)
# print(score_)

plt.plot(X_test, y_test, label=r"$f(x) = x \sin(x)$", linestyle="dotted")
# plt.scatter(X_train, y_train, label="Observations")
plt.plot(X_test, mean_prediction, label="Mean prediction")
# plt.fill_between(
    # X.ravel(),
    # mean_prediction - 1.96 * std_prediction,
    # mean_prediction + 1.96 * std_prediction,
    # alpha=0.5,
    # label=r"95% confidence interval",
# )
# plt.legend()
# plt.xlabel("$x$")
# plt.ylabel("$f(x)$")
# _ = plt.title("Gaussian process regression on noise-free dataset")

plt.show()